In [1]:
import pandas as pd
import json
import re
import csv

In [2]:
# benign_whois_data.txt

!head "../data/raw/benign_whois_data.txt" 1


==> ../data/raw/benign_whois_data.txt <==
{"google.com": {"address": null, "city": null, "country": "US", "creation_date": [ "1997-09-15 04:00:00", "1997-09-15 07:00:00"    ], "dnssec": "unsigned", "domain_name": [ "GOOGLE.COM", "google.com"    ], "emails": [ "abusecomplaints@markmonitor.com", "whoisrequest@markmonitor.com"    ], "expiration_date": [ "2028-09-14 04:00:00", "2028-09-13 07:00:00"    ], "name": null, "name_servers": [ "NS1.GOOGLE.COM", "NS2.GOOGLE.COM", "NS3.GOOGLE.COM", "NS4.GOOGLE.COM", "ns1.google.com", "ns3.google.com", "ns4.google.com", "ns2.google.com"    ], "org": "Google LLC", "referral_url": null, "registrar": "MarkMonitor, Inc.", "state": "CA", "status": [ "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited", "clientTransferProhibited https://icann.org/epp#clientTransferProhibited", "clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited", "serverDeleteProhibited https://icann.org/epp#serverDeleteProhibited", "serverTransferProhibit

In [3]:
import pandas as pd
import json

b = '../data/raw/benign_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 3731


In [4]:
record_dict = {}
for item in records:
    name = [x for x in item.keys()][0]
    record_dict[name] = item[name]

In [5]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [6]:
benign_df = pd.DataFrame.from_dict(record_dict, orient='index')
benign_df.reset_index(inplace=True)

In [7]:
benign_df.head(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,billing_organization,billing_phone,admin_country_code,registrar_country_code,registrar_organization_id,registrar_street,registrant_handle,registrant_zip_code,billing_street,registrant_country_code
0,google.com,None,None,US,"[1997-09-15 04:00:00, 1997-09-15 07:00:00]",unsigned,"[GOOGLE.COM, google.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2028-09-14 04:00:00, 2028-09-13 07:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,facebook.com,1601 Willow Rd,Menlo Park,US,1997-03-29 05:00:00,unsigned,FACEBOOK.COM,"[abusecomplaints@registrarsafe.com, domain@fb....",2031-03-30 04:00:00,Domain Admin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,youtube.com,None,None,US,2005-02-15 05:13:12,unsigned,"[YOUTUBE.COM, youtube.com]","[abusecomplaints@markmonitor.com, whoisrequest...","[2023-02-15 05:13:12, 2023-02-15 00:00:00]",None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.options.display.max_rows = 4000
print(str(list(benign_df.columns)))

['index', 'address', 'city', 'country', 'creation_date', 'dnssec', 'domain_name', 'emails', 'expiration_date', 'name', 'name_servers', 'org', 'referral_url', 'registrar', 'state', 'status', 'updated_date', 'whois_server', 'zipcode', 'email', 'fax', 'phone', 'tech_email', 'tech_fax', 'tech_name', 'tech_org', 'tech_phone', 'domain__id', 'registrant_country', 'registrant_name', 'registrant_state_province', 'registrar_id', 'registrar_url', 'admin', 'admin_application_purpose', 'admin_city', 'admin_country', 'admin_email', 'admin_fax', 'admin_id', 'admin_nexus_category', 'admin_organization', 'admin_phone', 'admin_postal_code', 'admin_state_province', 'admin_street', 'registrant_application_purpose', 'registrant_city', 'registrant_email', 'registrant_fax', 'registrant_id', 'registrant_nexus_category', 'registrant_organization', 'registrant_phone', 'registrant_postal_code', 'registrant_street', 'registrar_email', 'registrar_phone', 'tech_application_purpose', 'tech_city', 'tech_country', 'te

#############

In [9]:
import pandas as pd
import json

b = '../data/raw/malicious_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 2415


In [10]:
error_list = []
for x in record_dict:
    if type(record_dict[x]) != dict:
        error_list.append(x)
        
for x in error_list:
    del record_dict[x]

In [11]:
mal_df = pd.DataFrame.from_dict(record_dict, orient='index')
mal_df.reset_index(inplace=True)

In [12]:
mal_df.sample(3)

,index,address,city,country,creation_date,dnssec,domain_name,emails,expiration_date,name,...,billing_organization,billing_phone,admin_country_code,registrar_country_code,registrar_organization_id,registrar_street,registrant_handle,registrant_zip_code,billing_street,registrant_country_code
1192,lnk.to,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1201,royalsocietypublishing.org,REDACTED FOR PRIVACY,REDACTED FOR PRIVACY,US,"[2008-05-21 17:18:19, 2008-05-21 17:18:00]",unsigned,"[ROYALSOCIETYPUBLISHING.ORG, royalsocietypubli...","[abuse@enom.com, ABUSE@ENOM.COM]","[2022-05-21 17:18:19, 2022-05-21 17:18:00]",REDACTED FOR PRIVACY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2869,or.ke,None,None,None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#########

In [13]:
(mal_df.columns == benign_df.columns)*1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [14]:
mal_df['malicious'] = 'True'
benign_df['malicious'] = 'False'

In [15]:
combined_df = pd.concat([mal_df,benign_df], axis=0)

In [16]:
len(combined_df['index'])

7362

In [17]:
import pandas as pd
import json

b = '../data/raw/malicious_whois_data.txt'

records = []
error_lines = []

# load the lines into a list, there are some error lines that needs to be dealt with
with open(b) as f: # open file
    for line in f.read().splitlines(): # for each line
        try:
            obj = json.loads(re.sub('.*}{', '{', line)) # load the line, which looks like: {"domain.name": {...values...}}
            records.append(obj)
        except:
            error_lines.append(line)
        
print(f"Number of line errors: {len(error_lines)}")  
print(f"Number of lines: {len(records)}")     


Number of line errors: 0
Number of lines: 2415


In [18]:
DomainRecord_cols = ['index', 'dnssec', 'name']

combined_df.loc[:,DomainRecord_cols].to_csv("../data/external/combined_whois_data.csv")

In [19]:
list_of_explode_columns = ['country', 'emails','organization','whois_server', 'domain_status', 'registrar', 
                           'name_servers']

In [20]:
for _col in list_of_explode_columns:
    combined_df.loc[:,['index',_col]].explode(_col).to_csv(
    f"../data/external/combined_whois_data_{_col}.csv")

In [21]:
!docker ps

CONTAINER ID   IMAGE                                     COMMAND                  CREATED       STATUS         PORTS                                                                                                                               NAMES
38e46715231e   docker.tigergraph.com/tigergraph:latest   "/bin/sh -c '/usr/sb…"   2 weeks ago   Up 7 minutes   0.0.0.0:9000->9000/tcp, :::9000->9000/tcp, 0.0.0.0:14240->14240/tcp, :::14240->14240/tcp, 0.0.0.0:14022->22/tcp, :::14022->22/tcp   tigergraph


In [ ]:
# To start tigergraph docker
# https://docs.tigergraph.com/tigergraph-server/current/getting-started/docker
#  docker run -d -p 14022:22 -p 9000:9000 -p 14240:14240 --name tigergraph --ulimit nofile=1000000:1000000 -v /Users/nb311848/Documents/mystuff/repos/artemis/data/external:/home/tigergraph/data -t docker.tigergraph.com/tigergraph:latest


In [22]:
import pyTigerGraph as tg

In [23]:
conn = tg.TigerGraphConnection()

In [29]:
print(conn.gsql('ls', options=[]))

---- Global vertices, edges, and all graphs
Vertex Types:
Edge Types:

Graphs:
- Graph Artemis(DomainRecord:v, DomainName:v, Country:v, City:v, Emails:v, Organisation:v, Nameserver:v, Registrar:v, DomainRecordStatus:v, WhoisServer:v, DomainRecord_DomainName:e, Country_City:e, DomainRecord_Country:e, DomainRecord_Emails:e, DomainRecord_Organisation:e, DomainRecord_Nameserver:e, DomainRecord_Registrar:e, DomainRecord_DomainRecordStatus:e, DomainRecord_WhoisServer:e)
Jobs:


JSON API version: v2
Syntax version: v2



In [37]:
# Clear the server - We want to reproduce...

print(conn.gsql('drop all', options=[]))

Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph Artemis, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE and GSE
Stopping GPE GSE
Successfully stopped GPE GSE in 0.005 seconds
Clearing graph store...
Successfully cleared graph store
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.087 seconds
Everything is dropped.


In [38]:
#open text file in read mode
text_file = open("../src/scripts/DBImportExport_Artemis.gsql", "r")
 
#read whole file to a string
artemis_graph_gsql = text_file.read()
 
#close file
text_file.close()
 
print(artemis_graph_gsql)

CREATE GRAPH Artemis()
CREATE SCHEMA_CHANGE JOB schema_change_job_AddExportedLocalVETypes_JT8B8XV85X78MVIO FOR GRAPH Artemis { 
  ADD VERTEX DomainRecord(PRIMARY_ID id STRING, dnssec STRING, name STRING, entropy FLOAT, BenignFlag BOOL) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true";
  ADD VERTEX DomainName(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX Country(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX City(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX Emails(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX Organisation(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX Nameserver(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX Registrar(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX DomainRecordStatus(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE";
  ADD VERTEX WhoisServer(PRIMARY_ID id STRIN

In [39]:
print(conn.gsql(artemis_graph_gsql, options=[]))

Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 30.449 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.070 seconds
The graph Artemis is created.
Successfully created schema change jobs: [schema_change_job_AddExportedLocalVETypes_JT8B8XV85X78MVIO].

Current graph version 0
Trying to add vertex DomainRecord.
Trying to add vertex DomainName.
Trying to add vertex Country.
Trying to add vertex City.
Trying to add vertex Emails.
Trying to add vertex Organisation.
Trying to add vertex Nameserver.
Trying to add vertex Registrar.
Trying to add vertex DomainRecordStatus.
Trying to add vertex WhoisServer.
Trying to add edge DomainRecord_DomainName.
Trying to add edge Country_City.
Trying to add edge DomainRecord_Country.
Trying to add edge DomainRecord_Emails.
Trying to add edge DomainRecord_Organisation.
Trying to add edge DomainRecord_Nameserver.
Trying to add edge DomainRecord_Registrar.
Trying to add edge DomainRecord_DomainRecordStatus.
Trying to add e

In [40]:
loading_job = """RUN LOADING JOB load_job_benign_whois_data{field} USING MyDataSource="/home/tigergraph/data/combined_whois_data{field}.csv\""""

In [41]:
print(conn.gsql(loading_job.format(field=""),graphname='Artemis', options=[]))

for x in list_of_explode_columns:
    print(conn.gsql(loading_job.format(field=f"_{x}"),graphname='Artemis', options=[]))




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_benign_whois_data, jobid: Artemis.load_job_benign_whois_data.file.m1.1649077908086
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_benign_whois_data.file.m1.1649077908086.log'

Job "Artemis.load_job_benign_whois_data.file.m1.1649077908086" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_benign_whois_data.file.m1.1649077908086" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+-----------------------------------------------------------------------------------------+
|                                     FILENAME |   LOADED LINES |   AVG SPEED |   DURATION|
|/home/tigergraph/data/combined_whois_data.csv |           7363 |     35 kl/s |     




[Tip: Use "CTRL + C" to stop displaying the loading status update, then use "SHOW LOADING STATUS jobid" to track the loading progress again]
[Tip: Manage loading jobs with "ABORT/RESUME LOADING JOB jobid"]
Starting the following job, i.e.
JobName: load_job_benign_whois_data_name_servers, jobid: Artemis.load_job_benign_whois_data_name_servers.file.m1.1649077946697
Loading log: '/home/tigergraph/tigergraph/log/restpp/restpp_loader_logs/Artemis/Artemis.load_job_benign_whois_data_name_servers.file.m1.1649077946697.log'

Job "Artemis.load_job_benign_whois_data_name_servers.file.m1.1649077946697" loading status
[WAITING] m1 ( Finished: 0 / Total: 0 )
Job "Artemis.load_job_benign_whois_data_name_servers.file.m1.1649077946697" loading status
[FINISHED] m1 ( Finished: 1 / Total: 1 )
[LOADED]
+------------------------------------------------------------------------------------------------------+
|                                                  FILENAME |   LOADED LINES |   AVG SPEED |   DUR